In [2]:
import onnxruntime as ort
ort.get_device()

'GPU'

In [3]:
import onnx
import numpy
from skl2onnx.helpers import collect_intermediate_steps, compare_objects
from timeit import timeit
import onnxruntime as rt
from onnxconverter_common.data_types import FloatTensorType
from skl2onnx import convert_sklearn, __version__
import numpy as np
import pandas as pd

from sklearn import datasets, __version__ as skl_version
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

logistic = LogisticRegression()
pca = PCA()
pipe = Pipeline(steps=[('pca', pca), ('logistic', logistic)])

digits = datasets.load_digits()
X_digits = digits.data[:1000]
y_digits = digits.target[:1000]

pipe.fit(X_digits, y_digits)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:459: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


Pipeline(memory=None,
     steps=[('pca', PCA(copy=True, iterated_power='auto', n_components=None, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)), ('logistic', LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False))])

### CPU Execution env

In [4]:
initial_types = [('input', FloatTensorType((None, X_digits.shape[1])))]
model_onnx = convert_sklearn(pipe, initial_types=initial_types,
                             target_opset=12)

sess = rt.InferenceSession(model_onnx.SerializeToString(), providers=['CPUExecutionProvider'])
print("skl predict_proba")
print(pipe.predict_proba(X_digits[:2]))
onx_pred = sess.run(None, {'input': X_digits[:2].astype(np.float32)})[1]
df = pd.DataFrame(onx_pred)
print("onnx predict_proba")
print(df.values)

skl predict_proba
[[9.60391089e-01 2.32822583e-07 7.32337678e-04 1.46696345e-02
  5.59314384e-03 5.01685254e-04 7.49111880e-04 3.26559561e-03
  2.91410795e-04 1.38057590e-02]
 [2.00445372e-05 9.77887644e-01 5.29132862e-03 5.66014168e-04
  1.04206419e-02 1.53761203e-03 1.33897741e-03 1.31632792e-04
  2.68038846e-03 1.25716002e-04]]
onnx predict_proba
[[9.60390985e-01 2.00624967e-07 7.32338463e-04 1.46696689e-02
  5.59316622e-03 5.01648406e-04 7.49133644e-04 3.26562976e-03
  2.91393430e-04 1.38057200e-02]
 [2.00123977e-05 9.77887571e-01 5.29132457e-03 5.66035800e-04
  1.04206540e-02 1.53757061e-03 1.33896351e-03 1.31685068e-04
  2.68034847e-03 1.25733859e-04]]


In [5]:
compare_objects(pipe.predict_proba(X_digits[:2]), onx_pred)
# No exception so they are the same.

In [6]:
print("scikit-learn")
print(timeit("pipe.predict_proba(X_digits[:1])",
             number=10000, globals=globals()))
print("onnxruntime")
print(timeit("sess.run(None, {'input': X_digits[:1].astype(np.float32)})[1]",
             number=10000, globals=globals()))

scikit-learn
0.9679606409627013
onnxruntime
0.15207497600931674
